In [1]:
from pyspark.sql import SparkSession
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


spark = SparkSession.builder \
    .appName("Analyse_HIV") \
    .master("local[*]") \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.memory", "8g") \
    .config("spark.memory.fraction", "0.8") \
    .config("spark.memory.storageFraction", "0.3") \
    .getOrCreate()

Charger les données 

In [2]:
df = spark.read.load('new-cases-of-hiv-infection.csv', format='com.databricks.spark.csv',header='true',sep=',',inferSchema='true')

In [3]:
df.show(50)

+--------------------+----+----+---------------------------------------------------------+
|              Entity|Code|Year|Incidence - HIV/AIDS - Sex: Both - Age: All Ages (Number)|
+--------------------+----+----+---------------------------------------------------------+
|         Afghanistan| AFG|1990|                                                       88|
|         Afghanistan| AFG|1991|                                                      104|
|         Afghanistan| AFG|1992|                                                      120|
|         Afghanistan| AFG|1993|                                                      127|
|         Afghanistan| AFG|1994|                                                      133|
|         Afghanistan| AFG|1995|                                                      144|
|         Afghanistan| AFG|1996|                                                      152|
|         Afghanistan| AFG|1997|                                                      161|

In [4]:
df1 = spark.read.load('tb-patients-living-with-hiv-receiving-art.csv', format='com.databricks.spark.csv',header='true',sep=',',inferSchema='true')

In [5]:
DFA = spark.read.load('tb-patients-tested-positive-for-hiv.csv', format='com.databricks.spark.csv',header='true',sep=',',inferSchema='true')

In [6]:
df1.show(20)

+-----------+----+----+-------------------------------------------------+
|     Entity|Code|Year|TB patients living with HIV receiving ART - Total|
+-----------+----+----+-------------------------------------------------+
|Afghanistan| AFG|2007|                                                0|
|Afghanistan| AFG|2008|                                                0|
|Afghanistan| AFG|2009|                                                5|
|Afghanistan| AFG|2010|                                                2|
|Afghanistan| AFG|2011|                                                4|
|Afghanistan| AFG|2012|                                                5|
|Afghanistan| AFG|2015|                                                3|
|Afghanistan| AFG|2016|                                                8|
|    Albania| ALB|2003|                                                0|
|    Albania| ALB|2006|                                                3|
|    Albania| ALB|2007|               

In [7]:
from pyspark.sql.functions import col
df1 = df1.withColumnRenamed("TB patients living with HIV receiving ART - Total", "Tbpatients")

In [8]:
from pyspark.sql.functions import col
df1 = df1.withColumnRenamed("Entity", "Entit")
df1 = df1.withColumnRenamed("Code", "Cod")
df1 = df1.withColumnRenamed("Year", "Yea")
df1.show()

+-----------+---+----+----------+
|      Entit|Cod| Yea|Tbpatients|
+-----------+---+----+----------+
|Afghanistan|AFG|2007|         0|
|Afghanistan|AFG|2008|         0|
|Afghanistan|AFG|2009|         5|
|Afghanistan|AFG|2010|         2|
|Afghanistan|AFG|2011|         4|
|Afghanistan|AFG|2012|         5|
|Afghanistan|AFG|2015|         3|
|Afghanistan|AFG|2016|         8|
|    Albania|ALB|2003|         0|
|    Albania|ALB|2006|         3|
|    Albania|ALB|2007|         1|
|    Albania|ALB|2008|         4|
|    Albania|ALB|2009|         6|
|    Albania|ALB|2010|         0|
|    Albania|ALB|2011|         2|
|    Albania|ALB|2012|         7|
|    Albania|ALB|2013|         2|
|    Albania|ALB|2014|         2|
|    Albania|ALB|2015|         6|
|    Albania|ALB|2016|         6|
+-----------+---+----+----------+
only showing top 20 rows



In [9]:
from pyspark.sql.functions import col
df = df.withColumnRenamed("Incidence - HIV/AIDS - Sex: Both - Age: All Ages (Number)", "Incidence")
df.show()

+-----------+----+----+---------+
|     Entity|Code|Year|Incidence|
+-----------+----+----+---------+
|Afghanistan| AFG|1990|       88|
|Afghanistan| AFG|1991|      104|
|Afghanistan| AFG|1992|      120|
|Afghanistan| AFG|1993|      127|
|Afghanistan| AFG|1994|      133|
|Afghanistan| AFG|1995|      144|
|Afghanistan| AFG|1996|      152|
|Afghanistan| AFG|1997|      161|
|Afghanistan| AFG|1998|      165|
|Afghanistan| AFG|1999|      169|
|Afghanistan| AFG|2000|      178|
|Afghanistan| AFG|2001|      193|
|Afghanistan| AFG|2002|      208|
|Afghanistan| AFG|2003|      220|
|Afghanistan| AFG|2004|      233|
|Afghanistan| AFG|2005|      244|
|Afghanistan| AFG|2006|      252|
|Afghanistan| AFG|2007|      269|
|Afghanistan| AFG|2008|      282|
|Afghanistan| AFG|2009|      290|
+-----------+----+----+---------+
only showing top 20 rows



In [10]:
df_new = df.join(df1, df.Entity==df1.Entit)
df_new.show()

+-----------+----+----+---------+-----------+---+----+----------+
|     Entity|Code|Year|Incidence|      Entit|Cod| Yea|Tbpatients|
+-----------+----+----+---------+-----------+---+----+----------+
|Afghanistan| AFG|1990|       88|Afghanistan|AFG|2016|         8|
|Afghanistan| AFG|1990|       88|Afghanistan|AFG|2015|         3|
|Afghanistan| AFG|1990|       88|Afghanistan|AFG|2012|         5|
|Afghanistan| AFG|1990|       88|Afghanistan|AFG|2011|         4|
|Afghanistan| AFG|1990|       88|Afghanistan|AFG|2010|         2|
|Afghanistan| AFG|1990|       88|Afghanistan|AFG|2009|         5|
|Afghanistan| AFG|1990|       88|Afghanistan|AFG|2008|         0|
|Afghanistan| AFG|1990|       88|Afghanistan|AFG|2007|         0|
|Afghanistan| AFG|1991|      104|Afghanistan|AFG|2016|         8|
|Afghanistan| AFG|1991|      104|Afghanistan|AFG|2015|         3|
|Afghanistan| AFG|1991|      104|Afghanistan|AFG|2012|         5|
|Afghanistan| AFG|1991|      104|Afghanistan|AFG|2011|         4|
|Afghanist

In [11]:
from pyspark.sql.functions import col

# récupérer la liste des noms de colonnes du dataframe
cols = df_new.columns

# supprimer les colonnes 4, 5 et 6 du dataframe
cols_to_drop = [cols[4], cols[5], cols[6]]
df_ = df_new.drop(*cols_to_drop)
df_.show()

+-----------+----+----+---------+----------+
|     Entity|Code|Year|Incidence|Tbpatients|
+-----------+----+----+---------+----------+
|Afghanistan| AFG|1990|       88|         8|
|Afghanistan| AFG|1990|       88|         3|
|Afghanistan| AFG|1990|       88|         5|
|Afghanistan| AFG|1990|       88|         4|
|Afghanistan| AFG|1990|       88|         2|
|Afghanistan| AFG|1990|       88|         5|
|Afghanistan| AFG|1990|       88|         0|
|Afghanistan| AFG|1990|       88|         0|
|Afghanistan| AFG|1991|      104|         8|
|Afghanistan| AFG|1991|      104|         3|
|Afghanistan| AFG|1991|      104|         5|
|Afghanistan| AFG|1991|      104|         4|
|Afghanistan| AFG|1991|      104|         2|
|Afghanistan| AFG|1991|      104|         5|
|Afghanistan| AFG|1991|      104|         0|
|Afghanistan| AFG|1991|      104|         0|
|Afghanistan| AFG|1992|      120|         8|
|Afghanistan| AFG|1992|      120|         3|
|Afghanistan| AFG|1992|      120|         5|
|Afghanist

In [12]:
df2 = spark.read.load('tb-related-deaths-hiv.csv', format='com.databricks.spark.csv',header='true',sep=',',inferSchema='true')
df2.show()


+-----------+----+----+-------------------------------------------------------------------+
|     Entity|Code|Year|Estimated TB-related deaths among people living with HIV - estimate|
+-----------+----+----+-------------------------------------------------------------------+
|Afghanistan| AFG|2000|                                                            87.0184|
|Afghanistan| AFG|2001|                                                            74.7313|
|Afghanistan| AFG|2002|                                                            67.5944|
|Afghanistan| AFG|2003|                                                            69.5065|
|Afghanistan| AFG|2004|                                                            65.7143|
|Afghanistan| AFG|2005|                                                            64.4447|
|Afghanistan| AFG|2006|                                                            62.2394|
|Afghanistan| AFG|2007|                                                         

In [13]:
from pyspark.sql.functions import col
df2 = df2.withColumnRenamed("Entity", "Enti")
df2 = df2.withColumnRenamed("Code", "Co")
df2 = df2.withColumnRenamed("Year", "Ye")
df2 = df2.withColumnRenamed("Estimated TB-related deaths among people living with HIV - estimate", "TB-related")
df2.show()

+-----------+---+----+----------+
|       Enti| Co|  Ye|TB-related|
+-----------+---+----+----------+
|Afghanistan|AFG|2000|   87.0184|
|Afghanistan|AFG|2001|   74.7313|
|Afghanistan|AFG|2002|   67.5944|
|Afghanistan|AFG|2003|   69.5065|
|Afghanistan|AFG|2004|   65.7143|
|Afghanistan|AFG|2005|   64.4447|
|Afghanistan|AFG|2006|   62.2394|
|Afghanistan|AFG|2007|   60.0473|
|Afghanistan|AFG|2008|   66.4017|
|Afghanistan|AFG|2009|   76.5272|
|Afghanistan|AFG|2010|   80.4727|
|Afghanistan|AFG|2011|   89.6658|
|Afghanistan|AFG|2012|   97.8129|
|Afghanistan|AFG|2013|   102.495|
|Afghanistan|AFG|2014|   110.409|
|Afghanistan|AFG|2015|   105.271|
|Afghanistan|AFG|2016|    95.512|
|    Albania|ALB|2000|  0.155922|
|    Albania|ALB|2001|  0.146305|
|    Albania|ALB|2002|   0.16979|
+-----------+---+----+----------+
only showing top 20 rows



In [14]:
df_ne = df_.join(df2, df_.Entity==df2.Enti)
df_ne.show()

+-----------+----+----+---------+----------+-----------+---+----+----------+
|     Entity|Code|Year|Incidence|Tbpatients|       Enti| Co|  Ye|TB-related|
+-----------+----+----+---------+----------+-----------+---+----+----------+
|Afghanistan| AFG|1990|       88|         8|Afghanistan|AFG|2016|    95.512|
|Afghanistan| AFG|1990|       88|         8|Afghanistan|AFG|2015|   105.271|
|Afghanistan| AFG|1990|       88|         8|Afghanistan|AFG|2014|   110.409|
|Afghanistan| AFG|1990|       88|         8|Afghanistan|AFG|2013|   102.495|
|Afghanistan| AFG|1990|       88|         8|Afghanistan|AFG|2012|   97.8129|
|Afghanistan| AFG|1990|       88|         8|Afghanistan|AFG|2011|   89.6658|
|Afghanistan| AFG|1990|       88|         8|Afghanistan|AFG|2010|   80.4727|
|Afghanistan| AFG|1990|       88|         8|Afghanistan|AFG|2009|   76.5272|
|Afghanistan| AFG|1990|       88|         8|Afghanistan|AFG|2008|   66.4017|
|Afghanistan| AFG|1990|       88|         8|Afghanistan|AFG|2007|   60.0473|

In [15]:
from pyspark.sql.functions import col

# récupérer la liste des noms de colonnes du dataframe
cols = df_ne.columns

# supprimer les colonnes 4, 5 et 6 du dataframe
cols_to_drop = [cols[5], cols[6], cols[7]]
DF = df_ne.drop(*cols_to_drop)
DF.show()

+-----------+----+----+---------+----------+----------+
|     Entity|Code|Year|Incidence|Tbpatients|TB-related|
+-----------+----+----+---------+----------+----------+
|Afghanistan| AFG|1990|       88|         8|    95.512|
|Afghanistan| AFG|1990|       88|         8|   105.271|
|Afghanistan| AFG|1990|       88|         8|   110.409|
|Afghanistan| AFG|1990|       88|         8|   102.495|
|Afghanistan| AFG|1990|       88|         8|   97.8129|
|Afghanistan| AFG|1990|       88|         8|   89.6658|
|Afghanistan| AFG|1990|       88|         8|   80.4727|
|Afghanistan| AFG|1990|       88|         8|   76.5272|
|Afghanistan| AFG|1990|       88|         8|   66.4017|
|Afghanistan| AFG|1990|       88|         8|   60.0473|
|Afghanistan| AFG|1990|       88|         8|   62.2394|
|Afghanistan| AFG|1990|       88|         8|   64.4447|
|Afghanistan| AFG|1990|       88|         8|   65.7143|
|Afghanistan| AFG|1990|       88|         8|   69.5065|
|Afghanistan| AFG|1990|       88|         8|   6

In [16]:
from pyspark.sql.functions import col
DF = DF.withColumnRenamed("Incidence", "Incidence - VIH/SIDA - Sexe")
DF = DF.withColumnRenamed("Tbpatients", "TB patients living with HIV receiving ART")
DF = DF.withColumnRenamed("TB-related" , "Estimated TB-related deaths")
DF.show()


+-----------+----+----+---------------------------+-----------------------------------------+---------------------------+
|     Entity|Code|Year|Incidence - VIH/SIDA - Sexe|TB patients living with HIV receiving ART|Estimated TB-related deaths|
+-----------+----+----+---------------------------+-----------------------------------------+---------------------------+
|Afghanistan| AFG|1990|                         88|                                        8|                     95.512|
|Afghanistan| AFG|1990|                         88|                                        8|                    105.271|
|Afghanistan| AFG|1990|                         88|                                        8|                    110.409|
|Afghanistan| AFG|1990|                         88|                                        8|                    102.495|
|Afghanistan| AFG|1990|                         88|                                        8|                    97.8129|
|Afghanistan| AFG|1990| 

In [19]:
import pandas as pd
df_pd = pd.DataFrame(DF)
df_pd.to_csv('dij.csv', index=False)

ValueError: DataFrame constructor not properly called!

In [18]:
DF.printSchema()
DF.show()
DF.describe().show()

# Nettoyer les données
DF_clean = DF.dropna()
DF_clean = DF_clean.dropDuplicates()
DF_clean = DF_clean.filter("Year > 2010")

# Analyser les données
DF_populations = DF_clean.groupBy("Entity").agg({"Incidence - VIH/SIDA - Sexe": "mean", 
                                                   "TB patients living with HIV receiving ART": "sum",
                                                   "Estimated TB-related deaths": "sum"})

root
 |-- Entity: string (nullable = true)
 |-- Code: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Incidence - VIH/SIDA - Sexe: integer (nullable = true)
 |-- TB patients living with HIV receiving ART: integer (nullable = true)
 |-- Estimated TB-related deaths: double (nullable = true)

+-----------+----+----+---------------------------+-----------------------------------------+---------------------------+
|     Entity|Code|Year|Incidence - VIH/SIDA - Sexe|TB patients living with HIV receiving ART|Estimated TB-related deaths|
+-----------+----+----+---------------------------+-----------------------------------------+---------------------------+
|Afghanistan| AFG|1990|                         88|                                        8|                     95.512|
|Afghanistan| AFG|1990|                         88|                                        8|                    105.271|
|Afghanistan| AFG|1990|                         88|                             

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Jav

Py4JError: An error occurred while calling o100.showString

In [ ]:
from pyspark.sql.functions import to_date
# Afficher le schéma du dataframe
DF.printSchema()

# Convertir la colonne "year" en type "date"
DF = DF.withColumn("Year", to_date(DF["Year"].cast("string"), "yyyy"))

# Afficher le schéma du dataframe mis à jour
DF.printSchema()


In [ ]:
DF.show()

In [ ]:
nb_row = DF.count()
print("nombre total de ligne:", nb_row )

In [20]:
from pyspark.sql.functions import countDistinct

for col in DF.columns:
    num_distinct_values = DF.select(countDistinct(col)).collect()[0][0]
    print(f"{col}: {num_distinct_values}")


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving


Py4JError: An error occurred while calling o126.schema

Calcul du pourcentage de valeurs manquantes 

In [22]:
import matplotlib.pyplot as plt

# Calculer le pourcentage de valeurs manquantes pour chaque colonne
missing_percentages = DF.select([(sum(col(c).isNull().cast("int")) / df.count() * 100).alias(c) for c in df.columns])

# Transformer le résultat en Pandas DataFrame pour la visualisation
missing_percentages_pd = missing_percentages.toPandas().transpose()
missing_percentages_pd.columns = ['% de valeurs manquantes']

# Afficher le diagramme à barres
missing_percentages_pd.plot(kind='bar', figsize=(12,6))
plt.title('Pourcentage de valeurs manquantes par colonne')
plt.xlabel('Colonnes')
plt.ylabel('% de valeurs manquantes')
plt.show()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:32883)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:32883)